In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
%env JAX_TRACEBACK_FILTERING=off
import jax
jax.config.update('jax_traceback_filtering', 'off')


env: JAX_TRACEBACK_FILTERING=off


In [4]:
from sprint.icl_sfc_utils import Circuitizer

In [5]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [7]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [8]:
def check_if_single_token(token):
    return len(tokenizer.tokenize(token)) == 1

task_name = "antonyms"

task = tasks[task_name]

print(len(task))

# task = {
#     k:v for k,v in task.items() if check_if_single_token(k) and check_if_single_token(v)
# }

print(len(task))

pairs = list(task.items())

batch_size = 1 
n_shot=16
max_seq_len = 128
seed = 10

prompt = "Follow the pattern:\n{}"

runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

162
162


In [9]:
layers = list(range(6, 17))
circuitizer = Circuitizer(llama, tokenizer, runner, layers, prompt)

Setting up masks...


Running metrics...
Setting up RMS...


  0%|          | 0/18 [00:00<?, ?it/s]

Loading SAEs...


  0%|          | 0/11 [00:00<?, ?it/s]

Running node IEs...


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:

layers = [10,11,12,13,14,15,16]
orig_metric = circuitizer.ablated_metric(llama).tolist()
zero_metric = circuitizer.run_ablated_metrics([100000], layers=layers)[0][0]

print(orig_metric, zero_metric)

  0%|          | 0/1 [00:00<?, ?it/s]

-14.5625 -74.0


In [20]:
import numpy as np
# thresholds = np.linspace(0, 1e-4, 100)
# thresholds = np.linspace(1.4 * 1e-4, 1.45 * 1e-4, 200)
thresholds = np.logspace(-5, 0, 150)
topks = [4, 6, 12, 16, 24, 32]

inverse = False
do_abs = False
mean_ablate = False
average_over_positions = False


ablated_metrics, n_nodes_counts = circuitizer.run_ablated_metrics(thresholds, inverse=inverse, 
                                                                  do_abs=do_abs, mean_ablate=mean_ablate, 
                                                                  average_over_positions=average_over_positions,
                                                                  token_prefix=None, layers=layers)

faithfullness = np.array(ablated_metrics)
faithfullness = (faithfullness - zero_metric) / (orig_metric - zero_metric)



# target_metric = (max(ablated_metrics) - min(ablated_metrics)) * 0.95 + min(ablated_metrics)
# target_threshold = [threshold for threshold, metric in reversed(list(zip(thresholds, ablated_metrics))) if metric > target_metric][0]

  0%|          | 0/150 [00:00<?, ?it/s]

In [21]:
import matplotlib.pyplot as plt
import plotly.express as px

# plt.plot([max(n_nodes_counts) - x for x in n_nodes_counts], ablated_metrics)
# plt.plot(thresholds, ablated_metrics)
# plt.plot(thresholds, ablated_metrics)
# plt.plot(n_nodes_counts, ablated_metrics)
# plt.plot(thresholds, n_nodes_counts)
# plt.xscale("log")
# plt.plot(n_nodes_counts)

# px.line(x=list(range(len(ablated_metrics))), y=ablated_metrics)
# fig = px.line(x=thresholds, y=ablated_metrics)
# fig.update_xaxes(type="log", exponentformat="power")

fig = px.line(x=n_nodes_counts, y=faithfullness, title=f"inverse={inverse}, abs={do_abs}, mean={mean_ablate}, aop={average_over_positions}, layers={layers}")
fig.update_xaxes(title="Number of nodes")
fig.update_yaxes(title="Faithfullness")

fig


In [23]:
target_faithfullness = 0.3

target_threshold = [threshold for threshold, metric in reversed(list(zip(thresholds, faithfullness))) if metric > target_faithfullness][0]

target_threshold

0.9256417889712791

In [24]:
from tqdm.auto import tqdm

# layers = circuitizer.layers
# layers = [15,16]
selected_threshold = target_threshold


ablation_masks = {}

for layer in tqdm(layers):
    mask_attn_out, _ = circuitizer.mask_ie(circuitizer.ie_attn[layer], selected_threshold, None, do_abs=do_abs, average_over_positions=average_over_positions, inverse=inverse)
    mask_resid, _ = circuitizer.mask_ie(circuitizer.ie_resid[layer], selected_threshold, None, do_abs=do_abs, average_over_positions=average_over_positions, inverse=inverse)

    # print(mask_resid["arrow"].shape)

    # break

    try:
        mask_transcoder, _ = circuitizer.mask_ie(circuitizer.ie_transcoder[layer], selected_threshold, None, do_abs=do_abs, average_over_positions=average_over_positions, inverse=inverse)
    except KeyError:
        mask_transcoder = None

    ablation_masks[layer] = {
        "attn_out": mask_attn_out,
        "resid": mask_resid,
        "transcoder": mask_transcoder
    }

  0%|          | 0/7 [00:00<?, ?it/s]

In [25]:
circuit_nodes = []
n_nodes = 0

for layer, masks in ablation_masks.items():
    for mask_type, mask in masks.items():
        if mask is not None:
            for token_type, mask in mask.items():
                    n_nodes += mask.sum()
                    
                    node_ids = np.where(mask)

                    if len(node_ids) ==2:
                        for pos, feat in zip(*node_ids):
                            circuit_nodes.append((layer, mask_type, token_type, feat, pos))
                    else:
                        for feat in node_ids[0]:
                            circuit_nodes.append((layer, mask_type, token_type, feat, None))
                    

n_nodes

<jax.Array(224, dtype=int32)>

In [26]:
typed_ies = {
    "r": circuitizer.ie_resid,
    "a": circuitizer.ie_attn,
    "t": circuitizer.ie_transcoder,
}

circuit_nodes_with_ies = []

for node in circuit_nodes:
    layer, sae_type, token_type, node_id, pos = node
    ies = typed_ies[sae_type[0]][layer]

    if average_over_positions:
        masked_ies = circuitizer.mask_average(ies, token_type, average_over_positions=True)
        circuit_nodes_with_ies.append((*node, masked_ies[node_id].tolist()))
    else:
        masked_ies = circuitizer.mask_average(ies, token_type, average_over_positions=False)
        circuit_nodes_with_ies.append((*node, masked_ies[pos, node_id].tolist()))

circuit_nodes_with_ies = sorted(circuit_nodes_with_ies, key=lambda x: x[-1], reverse=True)

In [27]:
circuit_nodes_with_ies[:10]

[(16, 'resid', 'arrow', 10501, 15, 5.9219489097595215),
 (10, 'resid', 'output', 24942, 8, 4.2387542724609375),
 (11, 'resid', 'output', 5971, 8, 4.045263290405273),
 (16, 'resid', 'arrow', 20746, 56, 4.0150885581970215),
 (10, 'resid', 'input', 16643, 19, 3.845968723297119),
 (16, 'resid', 'arrow', 18817, 60, 3.39139986038208),
 (16, 'resid', 'arrow', 9435, 20, 3.25836443901062),
 (12, 'resid', 'output', 18089, 8, 3.1803691387176514),
 (13, 'resid', 'arrow', 12773, 7, 3.0938618183135986),
 (16, 'resid', 'arrow', 19087, 24, 3.0227627754211426)]

In [35]:
from tqdm.auto import tqdm
import numpy as np

combined_ies = {}

if average_over_positions:
    for node in circuit_nodes_with_ies:
        layer, type, mask, idx, pos, ie = node
        combined_ies[(layer, mask, type[0], idx)] = ie

else:
    for node in circuit_nodes_with_ies:
        layer, type, mask, idx, pos, ie = node
        combined_ies[(layer, mask, type[0], idx, pos)] = ie

In [36]:
combined_ies = [
    key + (weight,)
    for key, weight in combined_ies.items()
]

In [37]:
typed_ies_error = {
    "er": circuitizer.ie_error_resid,
    "ea": circuitizer.ie_error_attn,
    "et": circuitizer.ie_error_transcoder,
}

for layer in tqdm(layers):
    for type in typed_ies_error:
        if layer in typed_ies_error[type]: 
            ies = typed_ies_error[type][layer]
            for mask in circuitizer.masks:
                ies_mask = circuitizer.mask_average(ies, mask, average_over_positions=average_over_positions)
                # print(ies_mask.tolist())
                # raise

                if average_over_positions:
                    combined_ies.append((layer, mask, type, 0, ies_mask.tolist()))

                else:
                    for pos, ie in enumerate(ies_mask):
                        if ie > selected_threshold:
                            combined_ies.append((layer, mask, type, 0, pos, ie))

  0%|          | 0/7 [00:00<?, ?it/s]

In [38]:
combined_ies = sorted(combined_ies, key=lambda x: -x[-1])

In [39]:

from collections import defaultdict
circuit_node_dict = defaultdict(list)

if average_over_positions:
    for node in combined_ies:
        layer, mask, type, idx, weight = node
        circuit_node_dict[(type, layer, mask)].append(idx)

    circuit_node_dict = {
        k: np.array(v) for k,v in circuit_node_dict.items()
    }
else:
    for node in combined_ies:
        layer, mask, type, idx, pos, weight = node
        circuit_node_dict[(type, layer, mask)].append((pos, idx))

    circuit_node_dict = {
        k: np.array(v) for k,v in circuit_node_dict.items()
    }

In [40]:
list(circuit_node_dict.values())[:50]

[array([[64,  0],
        [44,  0],
        [24,  0],
        [60,  0],
        [11,  0],
        [48,  0],
        [36,  0],
        [56,  0],
        [28,  0],
        [40,  0],
        [52,  0],
        [ 7,  0],
        [68,  0],
        [20,  0],
        [15,  0],
        [32,  0]]),
 array([[ 8,  0],
        [16,  0],
        [12,  0],
        [37,  0],
        [45,  0],
        [25,  0],
        [33,  0],
        [29,  0],
        [57,  0],
        [53,  0],
        [65,  0],
        [61,  0],
        [21,  0]]),
 array([[44,  0],
        [11,  0],
        [60,  0],
        [48,  0],
        [36,  0],
        [56,  0],
        [24,  0],
        [64,  0],
        [32,  0],
        [28,  0],
        [15,  0],
        [68,  0],
        [52,  0],
        [ 7,  0],
        [20,  0],
        [40,  0]]),
 array([[ 8,  0],
        [16,  0],
        [25,  0],
        [21,  0],
        [33,  0],
        [29,  0],
        [41,  0],
        [57,  0]]),
 array([[16,  0],
        [ 8,  0],
        [25,  0],
        [12,  0],
        [33,  0],
        [29,  0],
        [21,  0],
        [61,  0],
        [49,  0],
        [57,  0],
        [37,  0]]),
 array([[64,  0],
        [44,  0],
        [36,  0],
        [11,  0],
        [60,  0],
        [28,  0],
        [48,  0],
        [52,  0],
        [32,  0],
        [ 7,  0],
        [56,  0],
        [40,  0],
        [24,  0],
        [20,  0],
        [68,  0],
        [15,  0]]),
 array([[36,  0],
        [56,  0],
        [44,  0],
        [64,  0],
        [ 7,  0],
        [48,  0],
        [68,  0],
        [32,  0],
        [52,  0],
        [60,  0],
        [40,  0],
        [28,  0],
        [20,  0],
        [15,  0],
        [11,  0],
        [24,  0]]),
 array([[64,  0],
        [36,  0],
        [11,  0],
        [56,  0],
        [48,  0],
        [40,  0],
        [32,  0],
        [28,  0],
        [24,  0],
        [44,  0],
        [52,  0],
        [68,  0],
        [ 7,  0],
        [60,  0],
        [20,  0],
        [15,  0]]),
 array([[36,  0],
        [44,  0],
        [56,  0],
        [32,  0],
        [64,  0],
        [52,  0],
        [68,  0],
        [15,  0],
        [48,  0],
        [28,  0],
        [11,  0],
        [ 7,  0],
        [40,  0],
        [60,  0],
        [20,  0]]),
 array([[23,  0],
        [35,  0],
        [27,  0],
        [43,  0],
        [ 6,  0],
        [19,  0],
        [14,  0],
        [63,  0],
        [10,  0],
        [59,  0],
        [55,  0],
        [47,  0],
        [67,  0],
        [31,  0],
        [39,  0]]),
 array([[23,  0],
        [35,  0],
        [10,  0],
        [43,  0],
        [47,  0],
        [63,  0],
        [27,  0],
        [59,  0],
        [55,  0],
        [ 6,  0],
        [31,  0],
        [14,  0],
        [67,  0],
        [51,  0],
        [19,  0],
        [39,  0]]),
 array([[36,  0],
        [11,  0],
        [68,  0],
        [44,  0],
        [28,  0],
        [ 7,  0],
        [56,  0],
        [15,  0],
        [52,  0],
        [32,  0],
        [20,  0],
        [40,  0]]),
 array([[12,  0],
        [33,  0],
        [21,  0],
        [16,  0],
        [45,  0],
        [49,  0]]),
 array([[23,  0],
        [10,  0],
        [35,  0],
        [59,  0],
        [27,  0],
        [63,  0],
        [55,  0],
        [43,  0],
        [67,  0],
        [31,  0]]),
 array([[   15, 10501],
        [   56, 20746],
        [   60, 18817],
        [   20,  9435],
        [   24, 19087],
        [    7, 30570],
        [   32,  8409],
        [   52,  4554],
        [   36, 12406],
        [   44, 19087],
        [   11, 19087],
        [   68, 28783],
        [    7, 24991],
        [   68, 19087],
        [   44, 24991],
        [   60, 24991],
        [   36,  4569],
        [   64, 24991],
        [   64,  7584],
        [   44, 19233],
        [   15, 28783],
        [   60, 16396],
        [   15, 29506],
        [    7, 29818],
        [   64, 29506],
        [   68, 12062],


In [45]:
import jax.numpy as jnp
from tqdm.auto import trange

if average_over_positions:
    important_feats_masks = {}
    for mask in circuitizer.masks:
        important_feats_masks[mask] = [
            (type, layer, feat) for layer, f_mask, type, feat, _ in combined_ies if f_mask == mask
            ]


    flat_feats = defaultdict(list)
    for k, v in important_feats_masks.items():
        for type, layer, feat in v:
            flat_feats[(k, type, layer)].append(feat)


    graph = []

    batch_size = 16
    # k = 32
    for type, features in tqdm(sorted(flat_feats.items(), key=lambda x: (-x[0][-1], x[0][-2], x[0][-3]))):
        mask, feature_type, layer = type
        mask = jnp.array(list(circuitizer.masks.keys()).index(mask))
        for batch in trange(0, len(features), batch_size, postfix=str(type)):
            batch_features = features[batch:batch+batch_size]
            orig_length = len(batch_features)
            batch_features = batch_features + [0] * (batch_size - len(batch_features))
            feature_effectss = jax.vmap(lambda x: circuitizer.compute_feature_effects(feature_type, layer, x, mask, layer_window=1, position=None))(jnp.asarray(batch_features))
            # feature_effectss = circuitizer.compute_feature_effects(feature_type, layer, batch_features, mask, layer_window=1)
            top_effects = defaultdict(list)
            for key, featuress in feature_effectss.items():
                for elem, feature_effects in enumerate(featuress):
                    if elem >= orig_length:
                        continue
                    if feature_effects.ndim == 0:
                        top_effects[elem].append((float(feature_effects), key, 0))
                        continue

                    nodes_to_keep = circuit_node_dict.get(key, np.empty(0, dtype=np.int32))
                    effects = feature_effects[nodes_to_keep]
                    for idx, effect in zip(nodes_to_keep, effects):
                        top_effects[elem].append((float(effect), key, int(idx)))
            for elem, effects in top_effects.items():
                effects.sort(reverse=True)
                edges = effects
                graph.extend([(weight,  key + (upstream_feature,), (type[1], type[2], type[0], batch_features[elem],) ) for weight, key, upstream_feature in edges])
            


    combined_ies = [
        (type, layer, mask, idx, weight) for layer, mask, type, idx, weight in combined_ies
    ] 


    sorted_graph = sorted(graph, reverse=True, key=lambda x: x[0])

    n_nodes = sum(map(len, important_feats_masks.values()))
    k_connections = 4
    weight_threshold = sorted_graph[n_nodes * k_connections][0]

In [59]:
if not average_over_positions:
    important_feats_masks = {}
    for mask in circuitizer.masks:
        important_feats_masks[mask] = [
            (type, layer, feat, pos) for layer, f_mask, type, feat, pos, _ in combined_ies if f_mask == mask
            ]


    flat_feats = defaultdict(list)
    for k, v in important_feats_masks.items():
        for type, layer, feat, pos in v:
            flat_feats[(k, type, layer)].append((pos, feat))


    circuit_node_dict

    graph = []

    batch_size = 16
    # k = 32
    for type, features in tqdm(sorted(flat_feats.items(), key=lambda x: (-x[0][-1], x[0][-2], x[0][-3]))):
        mask, feature_type, layer = type
        mask = jnp.array(list(circuitizer.masks.keys()).index(mask))
        for batch in trange(0, len(features), batch_size, postfix=str(type)):
            batch_features = features[batch:batch+batch_size]
            orig_length = len(batch_features)
            batch_features = batch_features + [(0, 0)] * (batch_size - len(batch_features))
            feature_effectss = jax.vmap(lambda x: circuitizer.compute_feature_effects(feature_type, layer, x[1], mask, layer_window=1, position=x[0]))(jnp.asarray(batch_features))
            # feature_effectss = circuitizer.compute_feature_effects(feature_type, layer, batch_features, mask, layer_window=1)
            top_effects = defaultdict(list)
            for key, featuress in feature_effectss.items():
                nodes_to_keep = circuit_node_dict.get(key, np.empty((0, 2), dtype=np.int32))

                for elem, feature_effects in enumerate(featuress):
                    if elem >= orig_length:
                        continue
                    if feature_effects.ndim == 1:
                        for idx, _ in nodes_to_keep:
                            top_effects[elem].append((float(feature_effects[idx]), key, 0, idx))
                        continue
                    effects = feature_effects[nodes_to_keep[:, 0], nodes_to_keep[:, 1]]

                    for idx, effect in zip(nodes_to_keep, effects):
                        top_effects[elem].append((float(effect), key, int(idx[1]), int(idx[0])))

                    
            for elem, effects in top_effects.items():
                effects.sort(reverse=True)
                edges = effects
                graph.extend([(weight,  key + (upstream_feature,upos,), (type[1], type[2], type[0], batch_features[elem][1], batch_features[elem][0],) ) for weight, key, upstream_feature, upos in edges])
            


    combined_ies = [
        (type, layer, mask, idx, pos, weight) for layer, mask, type, idx, pos, weight in combined_ies
    ] 


    sorted_graph = sorted(graph, reverse=True, key=lambda x: x[0])

    n_nodes = sum(map(len, important_feats_masks.values()))
    k_connections = 4
    weight_threshold = sorted_graph[n_nodes * k_connections][0]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s, ('arrow', 'er', 16)]

  0%|          | 0/1 [00:00<?, ?it/s, ('input', 'er', 16)]

  0%|          | 0/1 [00:00<?, ?it/s, ('newline', 'er', 16)]

  0%|          | 0/1 [00:00<?, ?it/s, ('output', 'er', 16)]

...

  0%|          | 0/1 [00:00<?, ?it/s, ('input', 'r', 10)]

  0%|          | 0/1 [00:00<?, ?it/s, ('output', 'r', 10)]

  0%|          | 0/1 [00:00<?, ?it/s, ('prompt', 'r', 10)]

  0%|          | 0/1 [00:00<?, ?it/s, ('input', 't', 10)]

  0%|          | 0/1 [00:00<?, ?it/s, ('output', 't', 10)]

In [103]:
graph[:100]

[(0.0001558343501528725, ('er', 16, 'arrow', 0), ('a', 16, 'arrow', 26950)),
 (0.00014752325660083443, ('er', 16, 'input', 0), ('a', 16, 'arrow', 26950)),
 (3.1936295272316784e-05, ('r', 16, 'input', 471), ('a', 16, 'arrow', 26950)),
 (8.613686077296734e-06, ('r', 16, 'input', 28017), ('a', 16, 'arrow', 26950)),
 (8.129306479531806e-06, ('er', 16, 'output', 0), ('a', 16, 'arrow', 26950)),
 (7.495141744584544e-06, ('r', 16, 'input', 17106), ('a', 16, 'arrow', 26950)),
 (5.879063792235684e-06, ('r', 16, 'arrow', 20746), ('a', 16, 'arrow', 26950)),
 (5.4994566198729444e-06, ('r', 16, 'arrow', 5839), ('a', 16, 'arrow', 26950)),
 (3.4953022804984357e-06,
  ('r', 16, 'arrow', 16396),
  ('a', 16, 'arrow', 26950)),
 (3.3527530831634067e-06,
  ('r', 16, 'arrow', 19087),
  ('a', 16, 'arrow', 26950)),
 (3.223263320251135e-06, ('r', 16, 'input', 12554), ('a', 16, 'arrow', 26950)),
 (2.9139926027710317e-06, ('er', 16, 'prompt', 0), ('a', 16, 'arrow', 26950)),
 (2.6358959530625725e-06,
  ('r', 16, 'arrow', 17459),
  ('a', 16, 'arrow', 26950)),
 (2.1566766008618288e-06,
  ('r', 16, 'arrow', 28783),
  ('a', 16, 'arrow', 26950)),
 (1.6950267536230967e-06, ('r', 16, 'arrow', 1244), ('a', 16, 'arrow', 26950)),
 (1.6587461004746729e-06, ('r', 16, 'arrow', 5202), ('a', 16, 'arrow', 26950)),
 (1.639681158849271e-06, ('r', 16, 'arrow', 1701), ('a', 16, 'arrow', 26950)),
 (1.3064176300758845e-06, ('r', 16, 'arrow', 6812), ('a', 16, 'arrow', 26950)),
 (1.1397964954085182e-06,
  ('r', 16, 'prompt', 15125),
  ('a', 16, 'arrow', 26950)),
 (1.1271797575318487e-06,
  ('r', 16, 'arrow', 25295),
  ('a', 16, 'arrow', 26950)),
 (1.0179868468185305e-06,
  ('r', 16, 'arrow', 29506),
  ('a', 16, 'arrow', 26950)),
 (1.0162028729610029e-06,
  ('r', 16, 'arrow', 28898),
  ('a', 16, 'arrow', 26950)),
 (9.177642255053797e-07, ('r', 16, 'arrow', 4756), ('a', 16, 'arrow', 26950)),
 (8.987248065750464e-07, ('r', 16, 'arrow', 7451), ('a', 16, 'arrow', 26950)),
 (8.81115170159319e-07, ('r', 16, 'arrow', 5603), ('a', 16, 'arrow', 26950)),
 (8.209453312701953e-07, ('r', 16, 'arrow', 10304), ('a', 16, 'arrow', 26950)),
 (7.130325911930413e-07, ('er', 16, 'newline', 0), ('a', 16, 'arrow', 26950)),
 (7.079574402268918e-07, ('r', 16, 'arrow', 25826), ('a', 16, 'arrow', 26950)),
 (6.415176585505833e-07, ('r', 16, 'arrow', 23646), ('a', 16, 'arrow', 26950)),
 (6.250346586966771e-07, ('r', 16, 'arrow', 3284), ('a', 16, 'arrow', 26950)),
 (6.193672561494168e-07, ('r', 16, 'arrow', 2179), ('a', 16, 'arrow', 26950)),
 (5.233213187239016e-07, ('r', 16, 'arrow', 16628), ('a', 16, 'arrow', 26950)),
 (4.076847801570693e-07, ('r', 16, 'arrow', 10501), ('a', 16, 'arrow', 26950)),
 (3.8714713923582167e-07,
  ('r', 16, 'arrow', 10418),
  ('a', 16, 'arrow', 26950)),
 (3.383754005881201e-07, ('r', 16, 'arrow', 20743), ('a', 16, 'arrow', 26950)),
 (2.4965538614196703e-07,
  ('r', 16, 'arrow', 29881),
  ('a', 16, 'arrow', 26950)),
 (1.9361469583145663e-07, ('r', 16, 'arrow', 9172), ('a', 16, 'arrow', 26950)),
 (1.5244594919749943e-07, ('r', 16, 'arrow', 9435), ('a', 16, 'arrow', 26950)),
 (1.5207525905225339e-07,
  ('r', 16, 'arrow', 18817),
  ('a', 16, 'arrow', 26950)),
 (1.0644294690109746e-07,
  ('r', 16, 'arrow', 17563),
  ('a', 16, 'arrow', 26950)),
 (9.248356747093567e-08, ('r', 16, 'arrow', 24937), ('a', 16, 'arrow', 26950)),
 (2.845575863830163e-08, ('r', 16, 'arrow', 8409), ('a', 16, 'arrow', 26950)),
 (2.7529324597708182e-08,
  ('r', 16, 'arrow', 24991),
  ('a', 16, 'arrow', 26950)),
 (-6.614264691506833e-08, ('r', 16, 'arrow', 4554), ('a', 16, 'arrow', 26950)),
 (-7.316972983062442e-08,
  ('r', 16, 'arrow', 18949),
  ('a', 16, 'arrow', 26950)),
 (-9.243964882443834e-08,
  ('r', 16, 'arrow', 22357),
  ('a', 16, 'arrow', 26950)),
 (-9.458952376917296e-08, ('r', 16, 'arrow', 5445), ('a', 16, 'arrow', 26950)),
 (-1.1527151855261764e-07,
  ('r', 16, 'arrow', 32002),
  ('a', 16, 'arrow', 26950)),
 (-1.1668454646951432e-07,
  ('r', 16, 'arrow', 4569),
  ('a', 16, 'arrow', 26950)),
 (-3.021081

In [66]:
if average_over_positions:

    _graph = [
        (w, l, (*r[:-1], int(r[-1]))) for w, l, r in sorted_graph
    ]
else:
    _graph = [
        (w, l, (*r[:-2], int(r[-2]), int(r[-1]))) for w, l, r in sorted_graph
    ]

In [105]:
if average_over_positions:
    _combined_ies = [
        (type, layer, mask, int(idx), weight) for type, layer, mask, idx, weight in combined_ies
    ]
else:
    _combined_ies = [
        (type, layer, mask, int(idx), int(pos), weight) for type, layer, mask, idx, pos, weight in combined_ies
    ]

In [106]:
import json
with open(f"micrlhf-progress/graph-rebirth-{task_name}_faith_0.3.json", 'w') as f:
    json.dump({"edges": _graph, "nodes": _combined_ies, "threshold": weight_threshold}, f)

In [42]:
import json
with open("micrlhf-progress/all-graph-antonyms.json") as f:
    all_graph = json.load(f)

nodes = all_graph["nodes"]

nodes[:100]

[['a', 11, 'arrow', 4080, 0.7506580352783203],
 ['a', 10, 'arrow', 18907, 0.32885661721229553],
 ['ea', 8, 'prompt', 0, 0.25985220074653625],
 ['a', 11, 'arrow', 24084, 0.2539865970611572],
 ['a', 11, 'arrow', 7345, 0.2489675134420395],
 ['a', 10, 'output', 18907, 0.24180087447166443],
 ['a', 12, 'arrow', 6492, 0.21518652141094208],
 ['a', 13, 'arrow', 4745, 0.21424899995326996],
 ['a', 11, 'arrow', 3528, 0.21369224786758423],
 ['a', 10, 'arrow', 30196, 0.14977753162384033],
 ['a', 13, 'arrow', 5151, 0.13242927193641663],
 ['a', 11, 'arrow', 29382, 0.12703268229961395],
 ['a', 9, 'arrow', 30422, 0.12628766894340515],
 ['a', 11, 'arrow', 23914, 0.11989074945449829],
 ['a', 10, 'arrow', 31128, 0.11808712780475616],
 ['a', 12, 'arrow', 24815, 0.11454211920499802],
 ['a', 7, 'output', 16193, 0.11348624527454376],
 ['a', 6, 'prompt', 30905, 0.11110686510801315],
 ['a', 11, 'input', 24084, 0.11046935617923737],
 ['a', 11, 'input', 30551, 0.10848592221736908],
 ['a', 11, 'input', 25963, 0.10084160417318344],
 ['a', 8, 'output', 12423, 0.09670715034008026],
 ['a', 11, 'arrow', 132, 0.09597808867692947],
 ['a', 16, 'arrow', 26950, 0.09281069040298462],
 ['a', 13, 'arrow', 30355, 0.08647042512893677],
 ['a', 9, 'input', 30422, 0.08515353500843048],
 ['a', 11, 'arrow', 6975, 0.08458726853132248],
 ['a', 11, 'input', 3528, 0.08040756732225418],
 ['a', 13, 'arrow', 15548, 0.07793214917182922],
 ['a', 9, 'output', 20099, 0.0748707726597786],
 ['a', 12, 'arrow', 9137, 0.07463555783033371],
 ['a', 11, 'arrow', 24250, 0.07212164252996445],
 ['a', 7, 'prompt', 17961, 0.07165984809398651],
 ['a', 10, 'arrow', 31964, 0.07049548625946045],
 ['a', 13, 'arrow', 17840, 0.06974411755800247],
 ['a', 8, 'output', 29606, 0.06762855499982834],
 ['a', 12, 'arrow', 11334, 0.06740531325340271],
 ['a', 7, 'output', 24464, 0.0671708807349205],
 ['a', 9, 'arrow', 3699, 0.06692551076412201],
 ['a', 8, 'input', 29606, 0.06609345972537994],
 ['a', 9, 'input', 10121, 0.06532280892133713],
 ['a', 6, 'arrow', 32736, 0.06465433537960052],
 ['a', 11, 'output', 3528, 0.06426970660686493],
 ['a', 7, 'output', 3570, 0.06188572943210602],
 ['a', 13, 'arrow', 21151, 0.06109173968434334],
 ['a', 11, 'arrow', 15611, 0.061014723032712936],
 ['a', 8, 'input', 1102, 0.060930412262678146],
 ['a', 6, 'input', 29059, 0.06056225299835205],
 ['a', 8, 'newline', 32349, 0.06011004000902176],
 ['a', 8, 'arrow', 18444, 0.05934614688158035],
 ['a', 8, 'input', 23841, 0.05843329802155495],
 ['a', 10, 'arrow', 12147, 0.05832074582576752],
 ['er', 14, 'arrow', 0, 0.05777547508478165],
 ['a', 13, 'arrow', 6077, 0.05575604736804962],
 ['a', 7, 'newline', 28287, 0.05573801323771477],
 ['a', 11, 'arrow', 20935, 0.055556949228048325],
 ['a', 12, 'arrow', 11345, 0.05515478923916817],
 ['a', 8, 'output', 31566, 0.05456184223294258],
 ['a', 11, 'arrow', 4522, 0.053785908967256546],
 ['a', 11, 'input', 32219, 0.05317726731300354],
 ['a', 6, 'input', 32736, 0.05151086300611496],
 ['a', 12, 'arrow', 28019, 0.05129891633987427],
 ['a', 8, 'output', 28812, 0.05086845904588699],
 ['a', 6, 'arrow', 833, 0.050552014261484146],
 ['a', 7, 'arrow', 538, 0.05022735148668289],
 ['a', 7, 'arrow', 13497, 0.05014926940202713],
 ['a', 9, 'output', 10121, 0.04978646710515022],
 ['a', 13, 'arrow', 22100, 0.049388784915208817],
 ['a', 9, 'arrow', 11624, 0.049090392887592316],
 ['a', 8, 'newline', 4213, 0.0490826815366745],
 ['a', 10, 'arrow', 6051, 0.048936307430267334],
 ['a', 10, 'output', 30196, 0.048778388649225235],
 ['a', 12, 'arrow', 21407, 0.048734430223703384],
 ['a', 7, 'output', 31467, 0.04839739203453064],
 ['a', 12, 'arrow', 22504, 0.04804452508687973],
 ['a', 7, 'newline', 31766, 0.04803304746747017],
 ['a', 7, 'prompt', 13170, 0.04800529405474663],
 ['a', 6, 'arrow', 25473, 0.04800485819578171],
 ['a', 13, 'arrow', 2844, 0.04782572016119957],
 ['a', 10, 'arrow', 21817, 0.04742930829524994],
 ['er', 13, 'arrow', 0, 0.047115836292505264],
 ['a', 11, 'arrow', 21699, 0.0469721294939518],
 ['a', 13, 'arrow', 

In [41]:
important_feats_masks = {}
for mask in circuitizer.masks:
    important_feats_masks[mask] = [
        (type, layer, feat) for layer, f_mask, type, feat, _ in combined_ies if f_mask == mask
        ]

In [20]:
from collections import defaultdict
flat_feats = defaultdict(list)
for k, v in important_feats_masks.items():
    for type, layer, feat in v:
        flat_feats[(k, type, layer)].append(feat)

In [36]:
from tqdm.auto import trange
import jax.numpy as jnp
graph = []

batch_size = 16
k = 32
for type, features in tqdm(sorted(flat_feats.items(), key=lambda x: (-x[0][-1], x[0][-2], x[0][-3]))):
    mask, feature_type, layer = type
    mask = jnp.array(list(circuitizer.masks.keys()).index(mask))
    for batch in trange(0, len(features), batch_size, postfix=str(type)):
        batch_features = features[batch:batch+batch_size]
        orig_length = len(batch_features)
        batch_features = batch_features + [0] * (batch_size - len(batch_features))
        feature_effectss = jax.vmap(lambda x: circuitizer.compute_feature_effects(feature_type, layer, x, mask, layer_window=1))(jnp.asarray(batch_features))
        # feature_effectss = circuitizer.compute_feature_effects(feature_type, layer, batch_features, mask, layer_window=1)
        top_effects = defaultdict(list)
        for key, featuress in feature_effectss.items():
            for elem, feature_effects in enumerate(featuress):
                if elem >= orig_length:
                    continue
                if feature_effects.ndim == 0:
                    top_effects[elem].append((float(feature_effects), key, 0))
                    continue
                effects, indices = jax.lax.top_k(jnp.abs(feature_effects), k)
                for i, e in zip(indices.tolist(), effects.tolist()):
                    top_effects[elem].append((e, key, i))
        for elem, effects in top_effects.items():
            effects.sort(reverse=True)
            edges = effects[:k]
            graph.extend([(weight,  key + (upstream_feature,), (type[1], type[2], type[0], batch_features[elem],) ) for weight, key, upstream_feature in edges])
        
        # # edges = circuitizer.compute_edges(*feature, mask, layer_window=1)
        # graph.extend([(weight, feature, downstream_feature) for weight, _, downstream_feature in edges])
    
        # for feature in tqdm(batch_features):
        #     feature_effects = circuitizer.compute_feature_effects(feature_type, layer, feature, mask, layer_window=1)
        #     top_effects = []
        #     for key, features in feature_effects.items():
        #         if features.ndim == 0:
        #             top_effects.append((float(features), key, 0))
        #             continue
        #         effects, indices = jax.lax.top_k(jnp.abs(features), k)
        #         for i, e in zip(indices.tolist(), effects.tolist()):
        #             top_effects.append((e, key, i))
        #     top_effects.sort(reverse=True)
        #     edges = top_effects[:k]
            
        #     # edges = circuitizer.compute_edges(*feature, mask, layer_window=1)
        #     graph.extend([(weight, feature, downstream_feature) for weight, _, downstream_feature in edges])



# for mask, features in tqdm(important_feats_masks.items()):
#     for batch in trange(0, len(features), batch_size):
#         batch_features = features[batch:batch+batch_size]
        
        
#         for feature in tqdm(batch_features):
#             # edges = circuitizer.compute_edges(*feature, mask, layer_window=1)
#             graph.extend([(weight, feature, downstream_feature) for weight, _, downstream_feature in edges])


#     # for downstream_feature in tqdm(features):
#     #     edges = compute_edges(downstream_feature, mask, layer_window=1)
#     #     graph.extend([(weight, upstream_feature_key + (upstream_feature,), downstream_feature[:2] + (mask,) + downstream_feature[2:])
#     #                   for weight, upstream_feature_key, upstream_feature in edges])

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s, ('arrow', 'a', 16)]

dict_keys([('er', 16, 'arrow'), ('er', 16, 'input'), ('er', 16, 'newline'), ('er', 16, 'output'), ('er', 16, 'prompt'), ('r', 16, 'arrow'), ('r', 16, 'input'), ('r', 16, 'newline'), ('r', 16, 'output'), ('r', 16, 'prompt')]) [26950, 25539, 1085, 14538, 19232, 7123, 20911, 12264, 12539, 12413, 32120, 26574, 23636, 23114, 178, 12793]
('er', 16, 'arrow') (16,)
('er', 16, 'input') (16,)
('er', 16, 'newline') (16,)
('er', 16, 'output') (16,)
('er', 16, 'prompt') (16,)
('r', 16, 'arrow') (16, 32768)
('r', 16, 'input') (16, 32768)
('r', 16, 'newline') (16, 32768)
('r', 16, 'output') (16, 32768)
('r', 16, 'prompt') (16, 32768)
[(6.0595695686060935e-05, ('er', 16, 'input', 0), ('a', 16, 'arrow', 26950)), (4.51656014774926e-05, ('r', 16, 'prompt', 5241), ('a', 16, 'arrow', 26950)), (3.955366264563054e-05, ('r', 16, 'arrow', 29818), ('a', 16, 'arrow', 26950)), (3.92732436012011e-05, ('r', 16, 'arrow', 24991), ('a', 16, 'arrow', 26950)), (3.684913463075645e-05, ('er', 16, 'prompt', 0), ('a', 16, '

ZeroDivisionError: division by zero

In [37]:
combined_ies[0]

(11, 'arrow', 'a', 23594, 0.41276583075523376)

In [ ]:
sorted_graph = sorted(graph, reverse=True, key=lambda x: x[0])